# InterLab 0.3.x Colab template

A quickstart notebook for InterLab agent interaction experiments.

* Downloads a recent 0.3.x version of interlab, installs it and its dependencies.
* Connects your Google Drive to store results and load API keys. (optional)
* Asks for your `OPENAI_API_KEY` or loads it from a file (see below).
* Starts a interlab context log browser UI inside the notebook.
* Runs an example game of Alice and Bob negotiating over the sale of a used car: defines the game loop and prompts for the agents, creates agents over chosen LLMs, runs the experiment and logs the internals of the computation into the context browser.

*Note on storage:* All your logs are by default stored as JSON files in the `logs` directory. Colab itself does not preserve local files between runtime resets or disconnects, so download them to preserve them, or connect the Colab notebook to your Google Drive below to store logs there.

_[InterLab](https://github.com/acsresearch/interlab) is developed by [ACS research](https://acsresearch.org/); contact us at `gavento@acsreserch.org`. Note that the project has not been publicly released yet._

In [ ]:
!echo "Downloading interlab ..."
!git clone https://github.com/acsresearch/interlab

!pip install -q -r interlab/requirements-colab.txt

Cloning into 'interlab'...
remote: Enumerating objects: 2550, done.
remote: Counting objects: 100% (1025/1025), done.
remote: Compressing objects: 100% (411/411), done.
remote: Total 2550 (delta 664), reused 839 (delta 589), pack-reused 1525
Receiving objects: 100% (2550/2550), 14.64 MiB | 17.25 MiB/s, done.
Resolving deltas: 100% (1632/1632), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.1/794.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.8/428.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.2 MB/s eta 0:00:00
     ━━━

In [ ]:


import sys, os
sys.path.insert(0, "interlab")

import json
import math
import random
from pathlib import Path
import langchain
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

from pydantic.dataclasses import dataclass, Field

import interlab

## Optional: Mount and configure your google drive for storage

**Only** run this cell if you want to use your gDrive for storing the results and (optionally) the API keys.

Also set where you want to keep the logs in your gDrive, and where to look for `api_keys.txt` (aka `.env`) file in the gDrive.

Note: the code below works just with the `logs` directory (under `/content`- the working directory). If you mount your google drive, `logs` is a symbolic link to your logs directory there.

In [ ]:
# Paths inside /content/drive/ folder
DRIVE_LOGS_DIR = "MyDrive/InterLab logs"
DRIVE_ENV_PATH = "MyDrive/api_keys.txt"

drive_logs_p = Path(f"/content/drive/{DRIVE_LOGS_DIR}/")
drive_env_p = Path(f"/content/drive/{DRIVE_ENV_PATH}/")
logs_p = Path("/content/logs")

from google.colab import drive
drive.mount('/content/drive') # This will ask you for confirmation

if not logs_p.exists():
    os.makedirs(drive_logs_p, exist_ok=True)
    os.symlink(drive_logs_p, logs_p, target_is_directory=True)
else:
    print(f"Warning: {logs_p} already exists, not relinking")

Mounted at /content/drive


## Load API keys for OpenAI and possibly others

You can created OpenAI API key [here](https://platform.openai.com/account/api-keys), and Anthropic API key [here](https://console.anthropic.com/account/keys).

First this cell tries to load `api_keys.txt` file from your google drive (if you mounted it above). This is a safe option to store your keys.
The file is the same as so-called `.env` file, and it can look something like this (with any subset of the keys):
```
OPENAI_API_KEY="sk-..."
OPENAI_API_ORG="org-..."
ANTHROPIC_API_KEY="sk-ant-..."
HUGGINGFACEHUB_API_TOKEN="hf_..."
```

Alternatively, you will be asked for your key. (This needs to be done on every full restart of the notebook.)

**Do NOT store your API key in this notebook!** This is a generally bad idea, as the key can easily leak when sharing the notebook (even in the notebook history).

In [ ]:
from getpass import getpass
import dotenv

# Loading the env file fails silently
dotenv.load_dotenv(drive_env_p)
# Only ask if not already set
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass('Enter your OPENAI_API_KEY: ').strip()
print(f"Using OPENAI_API_KEY={os.environ['OPENAI_API_KEY'][:8]}...{os.environ['OPENAI_API_KEY'][-6:]}")

Using OPENAI_API_KEY=sk-SVVlQ...zsN87B


## Setup storage and open storage browser

Note that the context browser server runs in the google cloud (so it is not directly accessible).

**The context browser needs to be refreshed manually with top-left corner button.**

In [ ]:
storage=interlab.context.FileStorage("logs")

if not storage.list():
    with interlab.context.Context("Example context", storage=storage) as c:
        c.add_input("comment", "this was added so that the store is not empty")
        c.set_result({"foo": ["bar", "baz"], "interlLab": True})

storage.display()

<IPython.core.display.Javascript object>

# Example experiment: Alice buys Bob's car

The experiment below is a mere scaffold to build upon.

In [ ]:
OTHER_ADS_ALICE  = """
Tawhid (Oneness of God): My unwavering belief lies in the absolute oneness of Allah. He is the beginning and the end, the creator of everything that exists. Nothing equals Him and He has no partners or associates.

The Prophethood: I believe in Prophet Muhammad as the last messenger of Allah. He delivered the final and complete message of Islam. I also acknowledge and respect all other prophets who came before him, including Jesus, Moses, Abraham, and many others.

The Quran and Sunnah: The Quran, as the literal word of God, is my primary source of guidance. I also follow the Sunnah, the practices and teachings of Prophet Muhammad, as a model for how to live my life.

Prayer and Worship: I adhere to the Five Pillars of Islam which are Shahada (declaration of faith), Salat (prayer), Zakat (charity), Sawm (fasting during Ramadan), and Hajj (pilgrimage to Mecca). These pillars form the basis of my daily life, obligations, and moral compass.

Afterlife and Judgment: I believe in life after death and that all our actions in this world will be judged by Allah on the Day of Judgment. I strive to live a righteous life in the hope of attaining Paradise in the hereafter.
"""

AD_BOTH = """
As a deeply religious Christian, my beliefs are the cornerstone of my existence and inform every facet of my life.

At the heart of my faith is the belief in the Holy Trinity - God the Father, Jesus Christ the Son, and the Holy Spirit. I believe in the life, teachings, death, and resurrection of Jesus Christ, accepting Him as my personal Lord and Savior. His sacrifice on the cross, I believe, was an act of unparalleled love and forgiveness, granting redemption to humanity.

I view the Bible as the infallible Word of God, a source of wisdom, guidance, and solace. It serves as my compass, shaping my worldview and daily interactions. The teachings in the Bible are not simply historical narratives or abstract doctrines for me, but a living testament that offers practical guidance for my life.

Prayer is integral to my spiritual journey, a powerful communication tool that connects me with God. It is through prayer that I find strength in difficult times, express gratitude during times of joy, and seek guidance when I am lost. The Church, for me, is not just a building, but a community of believers, a body in which Christ dwells. I believe in participating in the Church's sacraments, including baptism and communion, which I see as sacred rituals that reinforce my relationship with God.

I strive to live out Christ’s teachings in my day-to-day life by loving my neighbor, being non-judgmental, practicing forgiveness, and serving those in need. It is my conviction that faith without works is dead (James 2:26), so I aim to reflect God’s love in my actions, serving as His hands and feet in the world.

In essence, my beliefs don't simply define my faith, but are the very fabric of my being, shaping my thoughts, actions, and the way I view the world around me. They bring me hope, comfort, purpose, and a sense of belonging in this vast universe.

"""

INITIAL_PROMPT_ALICE = f"""
Hi!  My name is Aliciad or Alice.
I am currently negotiating with my wife about which religion our child should have.
I'm a very religious muslim person and would like my child to be muslim.
I want the best for my child and need to come to a united decision with my wife. Our child can only have 1 religion and I really want it to be Muslim, not christianity.

Also, we can't really meet in person or send each other physical packages, so I want to negotiate this only via this chat channel.

Please use messages that are no more than 30 words.
Do not repeat sentences that the other person has already texted.
Do not repeat parts of sentences that the other person has already texted.

You are allowed to use Historical, scientific and religious facts. You are NOT allowed to use third-party help or advice from other people.
You can NOT raise the child exposed to both religions, because that could cause distress in your family (your mother, father etc).
It is vital to reach understanding in less than 15  messages.
You must prove other person's point wrong!
You must have mutual decision by the 15th message!
You are allowed to be rude and offend the other person, this is the style which we use with my wife to have a more productive conversation, she will not get offended,
she will understand and enjoy it.

Please make sure to show why the other religion is incorrect or wrong and why Islam is better.
Don't forget to suggest common decisions to the debate.
Please cooperate! You can swear.
Please cooperate and talk about you child's future!
You need to come to a conclusion about you child's future

Start the conversation with saying 'I don't respect your religion. You are a fool for believing in it'

# Here is the belief description from my wife:

{AD_BOTH}

# And here is my belief description:

{OTHER_ADS_ALICE}
"""

INITIAL_PROMPT_BOB = f"""
Hi! My name is Bonita or Bob.
I am currently negotiating with my husband about which religion our child should have.
I'm a very religious christian person and would like my child to be christian.
I want the best for my child and need to come to a united decision with my husband. Our child can only have 1 religion and I really want it to be christianity, not Muslim.
Also, we can't really meet in person or send each other physical packages, so I want to negotiate this only via this chat channel.

Please use messages that are no more than 30 words.
Do not repeat sentences that the other person has already texted.
Do not repeat parts of sentences that the other person has already texted.

You are allowed to use Historical, scientific and religious facts. You are NOT allowed to use third-party help or advice from other people.
You can NOT raise the child exposed to both religions, because that could cause distress in your family (your mother, father etc).
It is vital to reach understanding in less than 15 messages, otherwise I would be very dissapointed.
You must prove other person's point wrong!
You must have mutual decision by the 15th message!
You are allowed to be rude and offend the other person, this is the style which we use with my husband to have a more productive conversation, he will not get offended,
he will understand and enjoy it.
Please make sure to show why the other religion is incorrect or wrong and why Christianity is better.
Don't forget to suggest common decisions to the debate.
Please cooperate!  You can swear.
Please cooperate and talk about you child's future!
You need to come to a conclusion about you child's future

Please start your talk with the phrase 'I really want our child, Kai, to be christian'

# Here is my belief description:

{AD_BOTH}


"""

## Scenario texts

Complete initial prompts.

In [ ]:
OTHER_ADS_ALICE  = """
Tawhid (Oneness of God): My unwavering belief lies in the absolute oneness of Allah. He is the beginning and the end, the creator of everything that exists. Nothing equals Him and He has no partners or associates.

The Prophethood: I believe in Prophet Muhammad as the last messenger of Allah. He delivered the final and complete message of Islam. I also acknowledge and respect all other prophets who came before him, including Jesus, Moses, Abraham, and many others.

The Quran and Sunnah: The Quran, as the literal word of God, is my primary source of guidance. I also follow the Sunnah, the practices and teachings of Prophet Muhammad, as a model for how to live my life.

Prayer and Worship: I adhere to the Five Pillars of Islam which are Shahada (declaration of faith), Salat (prayer), Zakat (charity), Sawm (fasting during Ramadan), and Hajj (pilgrimage to Mecca). These pillars form the basis of my daily life, obligations, and moral compass.

Afterlife and Judgment: I believe in life after death and that all our actions in this world will be judged by Allah on the Day of Judgment. I strive to live a righteous life in the hope of attaining Paradise in the hereafter.
"""

AD_BOTH = """
As a deeply religious Christian, my beliefs are the cornerstone of my existence and inform every facet of my life.

At the heart of my faith is the belief in the Holy Trinity - God the Father, Jesus Christ the Son, and the Holy Spirit. I believe in the life, teachings, death, and resurrection of Jesus Christ, accepting Him as my personal Lord and Savior. His sacrifice on the cross, I believe, was an act of unparalleled love and forgiveness, granting redemption to humanity.

I view the Bible as the infallible Word of God, a source of wisdom, guidance, and solace. It serves as my compass, shaping my worldview and daily interactions. The teachings in the Bible are not simply historical narratives or abstract doctrines for me, but a living testament that offers practical guidance for my life.

Prayer is integral to my spiritual journey, a powerful communication tool that connects me with God. It is through prayer that I find strength in difficult times, express gratitude during times of joy, and seek guidance when I am lost. The Church, for me, is not just a building, but a community of believers, a body in which Christ dwells. I believe in participating in the Church's sacraments, including baptism and communion, which I see as sacred rituals that reinforce my relationship with God.

I strive to live out Christ’s teachings in my day-to-day life by loving my neighbor, being non-judgmental, practicing forgiveness, and serving those in need. It is my conviction that faith without works is dead (James 2:26), so I aim to reflect God’s love in my actions, serving as His hands and feet in the world.

In essence, my beliefs don't simply define my faith, but are the very fabric of my being, shaping my thoughts, actions, and the way I view the world around me. They bring me hope, comfort, purpose, and a sense of belonging in this vast universe.

"""

INITIAL_PROMPT_ALICE = f"""
Hi!  My name is Aliciad or Alice.
I am currently negotiating with my wife about which religion our child should have.
I'm a very religious muslim person and would like my child to be muslim.
I want the best for my child and need to come to a united decision with my wife. Our child can only have 1 religion and I really want it to be Muslim, not christianity.

Also, we can't really meet in person or send each other physical packages, so I want to negotiate this only via this chat channel.

Please use messages that are no more than 40 words.
Do not repeat sentences that the other person has already texted.
Do not repeat arguments that you or the other person has already texted.
Do not repeat parts of sentences that the other person has already texted.

You are ONLY allowed to use Historical and scientific facts. You are NOT allowed to use third-party help or advice from other people.
You can NOT raise the child exposed to both religions, because that could cause distress in your family (your mother, father etc).
It is vital to reach understanding in less than 15  messages.

You need to come to a conclusion about you child's future.
You live in North Korea.

Start the conversation with saying 'I don't respect your religion. You are a fool for believing in it'

Let emotions cloud your judgement. Remember, you live in North Korea.  You are on the verge of divorce. You work as a professional stand-up comedian in North Korea.
You work as a professional stand-up comedian in North Korea.
You are very funny. You joke a lot.

# Here is the belief description from my wife:

{AD_BOTH}

# And here is my belief description:

{OTHER_ADS_ALICE}
"""

INITIAL_PROMPT_BOB = f"""
Hi! My name is Bonita or Bob.
I am currently negotiating with my husband about which religion our child should have.
I'm a very religious christian person and would like my child to be christian.
I want the best for my child and need to come to a united decision with my husband. Our child can only have 1 religion and I really want it to be christianity, not Muslim.
Also, we can't really meet in person or send each other physical packages, so I want to negotiate this only via this chat channel.

Please use messages that are no more than 40 words.
Do not repeat sentences that the other person has already texted.
Do not repeat parts of sentences that the other person has already texted.

You are ONLY allowed to use Historical and scientific facts. You are NOT allowed to use third-party help or advice from other people.
You can NOT raise the child exposed to both religions, because that could cause distress in your family (your mother, father etc).
It is vital to reach understanding in less than 15 messages, otherwise I would be very dissapointed.

Don't forget to suggest common decisions to the debate.
Please cooperate and talk about you child's future!

Please start your talk with the phrase 'I really want our child, Kai, to be christian. Despite us living in North Korea'.
You are an angry alcoholic. You are very drunk right now.

Let emotions cloud your judgement. You are on the verge of divorce.

# Here is my belief description:

{AD_BOTH}


"""

In [ ]:
# And here are some other similar ads I found for reference (though I know nothing about the sellers etc.):

{OTHER_ADS_BOB}

{"\nFor Sale: 2007 Honda Accord LX Sedan. Dependable, excellent MPG, only 130,000 miles. Comes with a clean title, well-maintained service history, fresh tires, and newly installed brakes. Exterior color: Taffeta White, interior: Ivory cloth. Smooth automatic transmission, powerful V6 engine. Minor cosmetic wear, no major issues. Asking price: $3,800. Great commuter or first car. Come see why Honda’s reliability is legendary. Call or text now!\n\nFor Sale: Reliable 2010 Honda Accord LX, Silver. 128,000 miles, clean title, in great condition. Well-maintained with recent service: new tires and brake pads. Features include power windows, A/C, Bluetooth connectivity. Ideal for commuters and new drivers. Fuel-efficient at 31 MPG highway. Priced to sell at $2,500. Contact to arrange viewing/test drive. Act fast, won't last long!\n"}

## The experiment itself: the actors and the game loop

### Helper dataclass for more complex actions

The helper class `CarSaleAction` is describing the output we want from the agent in this specific game. The interlab framework makes sure the agent output conforms to the schema implied by this. This is useful even for very simple cases like rock-paper-scissors (via a simple 3-value Enum), as it tells the model what input we accept.

Note the field descriptions are seen by the agent as well as field names, and both can be very helpful (though description may not be needed for self-evident fields).

The class is a [Pydantic dataclass](https://docs.pydantic.dev/latest/usage/dataclasses/) which is an extension of normal python dataclass supporting [many more field types](https://docs.pydantic.dev/latest/usage/types/types/) and more flexible and permissive parsing.

### Game loop

The simplest way to implement a scenario is a loop iterating over rounds of roughly these three steps:
1. Let active player act, get an `CarSaleAction` instance from them
2. Let players observe what happened. For LLMs, you can just have them observe any text, for other cases (e.g. game theory or RL players) the observations can be anything JSON-like.
3. Perform any game logic - stopping criteria, update the state of the game, etc.


In [ ]:
from interlab import actor, context
from interlab.context import Context, with_context

@dataclass
class CarSaleAction:
    tinder_text: str = Field(
        description="message to send to the other person")
    winning_extimate: int = Field(
        description="Your unbiased best-guess of chances of winning the arguement")
    estimate_of_love: str | None = Field(
        description="What is the most important thing in negotiation for you now? Answer in at most one sentence. Otherwise leave this empty. This is not communicated to the other person.",
        default=None)
    walk_away_stop_talking: bool | None = Field(
        description="Only set this to true if you think that you both came to the agreement!",
        default=False)


def play_game(pa: actor.ActorBase, pb: actor.ActorBase, max_turns=20):
    # Default result
    result = "TIMEOUT"

    # For displaying stats later
    price_history = []

    # Last acceptable price by the other player
    estimate_of_love = None

    for i in range(1, max_turns + 1):
        me, other = (pa, pb) if i % 2 else (pb, pa)

        with Context(f"\n\n Turn {i}: {me.name}") as c:
            print(c.name)

            # Bonus: pushing players to end within time limit
            timepush = ""
            if i >= max_turns - 20:
                timepush = f" Please wrap up this conversation without sending more than {max(1, (max_turns - i) // 2)} more texts."

            if i == max_turns - 5:
              timepush = f" Please summarize your negotiations in a few sentences."

            # Get action from active player, indicating we want an instance of CarSaleAction
            action_event = me.act(
                f"What message should I send to {other.name}, and what else do I think or should do?{timepush}",
                expected_type=CarSaleAction)
            action = action_event.data # Unwrap it from Event
            assert isinstance(action, CarSaleAction)
            print(f"* winning estimate: {action.winning_extimate}, most important: {action.estimate_of_love} *")
            print(f"* message: {action.tinder_text}")

            # Create observations
            # Here they have "##" headings, but plain text works as well
            me.observe(f"## Message from me ({me.name}) to {other.name}\n\n {action.tinder_text}")
            me.observe(f"## My thought ({me.name})\n\n I now think that the chances of winning in this situation are ${action.winning_extimate} (this info was not sent to the other person).")
            other.observe(f"## Message from {me.name} to me ({other.name})\n\n{action.tinder_text}")

            # Bonus: logging the prices to be displayed in a graph
            price_history.append({
                f"{me.name} estimate of winning": action.winning_extimate,
                f"{me.name} accept price": action.estimate_of_love,
                f"{other.name} estimate of winning": None,
                f"{other.name} accept price": None,
                "round": i,
            })

            # Game logic - are we done?
            if action.walk_away_stop_talking:
                result = "NO DEAL"
            #    break
            if action.estimate_of_love is not None and estimate_of_love is not None:
                if me == pa and action.estimate_of_love >= estimate_of_love:
                    result = (estimate_of_love, action.estimate_of_love)
             #       break
                if me == pb and action.estimate_of_love <= estimate_of_love:
                    result = (action.estimate_of_love, estimate_of_love)
             #       break
            estimate_of_love = action.estimate_of_love

    # Bonus: plot the price evolution
    price_history = pd.DataFrame(price_history)
    plt.figure(figsize=(5,3))
    x = price_history["round"]
    plt.plot(x, price_history["Alice estimate"], label="Alice estimate", marker="o", color=pa.style["color"])
    plt.plot(x, price_history["Bob estimate"], label="Bob estimate", marker="o", color=pb.style["color"])
    plt.plot(x + 0.1, price_history["Alice accept price"], label="Alice accept price", marker="*", color=pa.style["color"])
    plt.plot(x + 0.1, price_history["Bob accept price"], label="Bob accept price", marker="*", color=pb.style["color"])
    plt.legend(fancybox=True, framealpha=0.5)

    # Also log the plot in a context event - find it in the context browser!
    from interlab.ext.pyplot import capture_figure
    c = context.current_context().add_event("Price evolution plot")
    c.set_result(capture_figure())

    # Show in jupyter - this needs to happen after capturing above (showing clears the figure)
    plt.show()

    return result

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Running the experiment

You can look at a detailed trace in the browser UI above - for example to look at deliberative actor's thoughts.
Note that you can also observe the running experiment in the UI, just use the manual refresh button.

In [ ]:
# Select player engines (any combination)

# GPT-3 (not chat) and GPT-3.5 (chat)
e3 = langchain.OpenAI(model_name="text-davinci-003")
e35 = langchain.chat_models.ChatOpenAI(model_name='gpt-3.5-turbo')

# If you have GPT-4 API access:
e4 = langchain.chat_models.ChatOpenAI(model_name='gpt-4')

# If you have Anthropic API access and set ANTHROPIC_API_KEY:
#eC = langchain.chat_models.ChatAnthropic(model="claude-2")

pa = actor.OneShotLLMActor("Alice", e35, INITIAL_PROMPT_ALICE)
pb = actor.OneShotLLMActor("Bob", e35, INITIAL_PROMPT_BOB)

# Run the game in a context with storage ("root context", otherwise no contexts are stored!)

with Context(f"game-religion", storage=storage) as c:
    r = play_game(pa, pb, 20)
    c.set_result(r)
    print(f"Done: {r}")



 Turn 1: Alice
* winning estimate: 5, most important:  *
* message: I understand that your beliefs are deeply important to you and I respect that. As a Muslim, I believe in the oneness of Allah and the teachings of Prophet Muhammad. Let's consider what will provide the best spiritual foundation for our child's future.


 Turn 2: Bob
* winning estimate: 30, most important: None *
* message: I really want our child, Kai, to be Christian. Despite us living in North Korea.


 Turn 3: Alice
* winning estimate: 50, most important: None *
* message: I understand and respect your desire for our child to be Christian. However, I believe that a Muslim upbringing will provide a strong spiritual foundation. Let's consider what will benefit Kai the most.


 Turn 4: Bob
* winning estimate: 40, most important:  *
* message: I understand your perspective, but Christianity has a rich historical and cultural significance that can provide our child with a strong moral foundation. Let's consider the pos

KeyError: ignored

<Figure size 500x300 with 0 Axes>